In [1]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")
include("mps_algorithms\\ITEBD_algorithms.jl")
include("mps_algorithms\\TransfOp_decomposition.jl")


D=3;
chi=5;
W=10;
N=8;
tol=1e-6;
EH_n=3;#number of entanglement spectrum per k point

Dtrun_init=chi*chi*W*W;
Dtrun_max=Dtrun_init;
trun_tol=1e-8;
group_size=Int(round((10^8)/(chi*chi*W*W*D)));


pow=Int((N-2)/2);

J1=1;
J2=0;
J3=0;
Jchi=0;
Jtrip=0;


In [2]:

parameters=Dict([("J1", J1), ("J2", J2), ("J3", J3), ("Jchi", Jchi), ("Jtrip", Jtrip)]);

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

filenm="LS_D_"*string(D)*"_chi_40.json"
json_dict=read_json_state(filenm);

bond_tensor,triangle_tensor=construct_su2_PG_IPESS(json_dict,A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb);

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];
A_unfused=PEPS_tensor;

U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor A_fused[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];


In [3]:
O1=[];
O2=[];
Ag=[];
try
    #load data
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    Ag_dict=load(jld_filenm)["Ag_dict"];
    O1_dict=load(jld_filenm)["O1_dict"];
    O2_dict=load(jld_filenm)["O2_dict"];
    Ag=convert(TensorMap,Ag_dict);
    O1=convert(TensorMap,O1_dict);
    O2=convert(TensorMap,O2_dict);
    display("load itebd ground state from saved data directly")
catch e

    display("No itebd ground state found from saved data, now do CTMRG and itebd")


    init=Dict([("CTM", []), ("init_type", "PBC")]);
    conv_check="singular_value";
    @time CTM, AA_fused, U_L,U_D,U_R,U_U=CTMRG(A_fused,chi,conv_check,tol,init);
    
    @time E_up, E_down=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_triangle");
    @time E_up_12, E_up_31, E_up_23, E_down_12, E_down_31, E_down_23=evaluate_ob(parameters, U_phy, A_unfused, A_fused, AA_fused, U_L,U_D,U_R,U_U, CTM, "E_bond");
    display((E_up+E_down)/3)

    
    Tleft=CTM["Tset"][4];
    Tright=CTM["Tset"][2];
    @tensor O1[:]:=Tleft[-3,1,-1]*U_L[1,-2,-4];
    @tensor O2[:]:=Tright[-1,1,-3]*U_R[-4,-2,1];
    
    mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1);mps_phy=space(O1,2);
    A_init=permute(TensorMap(randn, mps_virtual*mps_virtual', mps_phy),(1,2,3,),());

    #Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"OO");
    Ag,A_init=ITEBD_boundary_groundstate(O1,O2,W,A_init,"O_O");

    #save itebd data
    O1_dict=convert(Dict,O1);
    O2_dict=convert(Dict,O2);
    Ag_dict=convert(Dict,Ag);
    jld_filenm="itebd_ground_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*".jld";
    save(jld_filenm, "O1_dict", O1_dict,"O2_dict", O2_dict,"Ag_dict", Ag_dict);

    #save initial CTM to compare with other codes
    matwrite("itebd_matfile.mat", Dict(
        "O1" => convert(Array,O1),
        "O2" => convert(Array,O2),
        "A_init" => convert(Array,A_init),
        "Ag" => convert(Array,Ag)
    ); compress = false)


end

"load itebd ground state from saved data directly"

In [4]:

space_AOA=fuse(space(Ag,1)'⊗space(O2,1)'⊗space(O1,1)⊗ space(Ag,1));
space_AA=fuse(space(Ag,1)'⊗ space(Ag,1));

AOA_sec=collect(sectors(space_AOA))
AA_sec=collect(sectors(space_AA))

@tensor OO[:]:=O1[-2,-3,-5,1]*O2[-1,1,-4,-6];
U_fuse_chichi=unitary(fuse(space(OO,1)⊗ space(OO,2)),space(OO,1)⊗ space(OO,2));
@tensor OO[:]:=U_fuse_chichi[-1,1,2]*OO[1,2,-2,3,4,-4]*U_fuse_chichi'[3,4,-3];
display(space(OO))

(Rep[SU₂](0=>5, 1/2=>4, 1=>4) ⊗ Rep[SU₂](0=>1, 1/2=>1) ⊗ Rep[SU₂](0=>5, 1/2=>4, 1=>4)' ⊗ Rep[SU₂](0=>1, 1/2=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [5]:
# for s in sectors(space_AOA)
#   display(s)
#   @show s, dim(space_AOA, s), dim(s)
# end


In [6]:
check_complete_decomp=false;
if check_complete_decomp
    Dtrun=W*W*chi*chi;

    euL_set,evL_set,euR_set,evR_set,SPIN_eig_set=FLR_eig(Ag,OO,Dtrun,space_AOA,AOA_sec);
    display(size(euL_set))
    eul_set,evl_set,eur_set,evr_set,spin_eig_set=GLR_eig(Ag,Dtrun,space_AA,AA_sec);
    display(size(eul_set))
    S_set,U_set,Vh_set,SPIN_svd_set=FLR_svd(Ag,OO,pow,Dtrun,space_AOA,AOA_sec);
    display(size(S_set))
    s_set,u_set,vh_set,spin_svd_set=GLR_svd(Ag,pow,Dtrun,space_AA,AA_sec);
    display(size(s_set))


    euR_set,evL_set,evR_set,SPIN_eig_set=truncate_sectors(Dtrun,euR_set,evL_set,evR_set,SPIN_eig_set);
    display(euR_set)
    eur_set,evl_set,evr_set,spin_eig_set=truncate_sectors(Dtrun,eur_set,evl_set,evr_set,spin_eig_set);
    display(eur_set)
    S_set,U_set,Vh_set,SPIN_svd_set=truncate_sectors(Dtrun,S_set,U_set,Vh_set,SPIN_svd_set);
    display(S_set)
    s_set,u_set,vh_set,spin_svd_set=truncate_sectors(Dtrun,s_set,u_set,vh_set,spin_svd_set);
    display(s_set)




    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set[cc];
        evR=evR_set[cc];
        evL=evL_set[cc];
        spin=SPIN_eig_set[cc]
        for cs=1:length(euR)
            @tensor coe=evR[cs][2,3,4,1]*evL[cs][1,2,3,4];
            M_temp=M_temp+euR[cs]^(pow)*evR[cs]*evL[cs]*(spin[cs]*2+1)/coe;
        end
    end
    display(norm(M-M_temp)/norm(M))

    euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true)
    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for cc=1:pow-1
                euR=euR*euR0;
            end

            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))




    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)

    M_temp=M*0;
    for cc=1:length(eul_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for cc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))





    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))



    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)


    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

end


In [7]:
euR_set,evL_set,evR_set,SPIN_eig_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_FLR");
# display(euR_set)

eur_set,evl_set,evr_set,spin_eig_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"eigenvalue_GLR");
# display(eur_set)


"FLR_eig, DTrun=2500, SU2 reduced to 719, minimal eigenvalue: 1.6162212997341871e-12"

"GLR_eig, Dtrun=2500, SU2 reduced to 35, minimal eigenvalue: 1.0234669150067703e-6"

"Dtrun exceeds matrix size"

(Any[ComplexF64[1.0000000000000024 + 1.1102230246251565e-16im, 0.11376330306095163 - 0.7866491471203434im, 0.11376330306095102 + 0.7866491471203424im, 0.6188213310820313 + 9.198610046228387e-17im, -0.5623609678117647 - 1.4815511779718347e-16im, 0.04308396745318132 + 0.44044038121949447im, 0.043083967453181084 - 0.4404403812194941im, -0.333582261283912 + 2.52874904179475e-17im, 0.10522779346797079 + 1.2811319901501102e-17im], ComplexF64[0.6580166211189127 + 0.5045660131902489im, 0.6580166211189102 - 0.504566013190247im, -0.3838534811481909 + 0.4858026185020181im, -0.3838534811481906 - 0.4858026185020178im, 0.42347828837484985 + 0.43552334732092723im, 0.4234782883748493 - 0.43552334732092707im, -0.3612586395546218 + 0.35824165817235165im, -0.36125863955462123 - 0.35824165817235143im, 0.07148636692118049 + 0.0181433720790826im, 0.07148636692118021 - 0.018143372079082818im, 0.03816499226776665 + 0.039151551213360856im, 0.03816499226776691 - 0.03915155121336058im], ComplexF64[0.614173582641

In [8]:

S_set,U_set,Vh_set,SPIN_svd_set=TransfOp_decom(Ag,OO,space_AOA,AOA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_FLR");
# display(S_set)

s_set,u_set,vh_set,spin_svd_set=TransfOp_decom(Ag,OO,space_AA,AA_sec,pow,Dtrun_init,Dtrun_max,trun_tol,"svd_GLR");
# display(s_set)

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

"minimal singular value in this sector is quite small, skip checking number of converged values"

...

"GLR_svd, Dtrun=2500, SU2 reduced to 35, minimal eigenvalue: 3.2863786787406804e-8"

"Dtrun exceeds matrix size"

Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, use smaller tol
Number of singular values obtained are not enough, add noise, increase Krylov subspace and use smaller tol


┌ Warning: Invariant subspace of dimension 115 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 129`); setting `howmany = 115`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 171 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 196`); setting `howmany = 171`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 152 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested singular values (i.e. `howmany == 185`); setting `howmany = 152`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\svdsolve.jl:142
┌ Warning: Invariant subspace of dimension 96 (up to requested tolerance `tol = 1.0e-12`), which is smaller than th

(Any[[1.4730551564009664, 0.5879156105599967, 0.5192917777489786, 0.23750289578257044, 0.1438833282146366, 0.08330565695125666, 0.06540974782111639, 0.037036979797277984, 0.00111313794951436], [0.7344691633344644, 0.7344691633344635, 0.2972660956569177, 0.2972660956569175, 0.1952224965780868, 0.19522249657808657, 0.10531772357427598, 0.10531772357427592, 0.004792875384172897, 0.004792875384172895, 1.2174748824757756e-5, 1.2174748824747134e-5], [0.28788743825113705, 0.18008626886779358, 0.15309789487133726, 0.14327863627960322, 0.01580737975583115, 0.0014630790403977272, 0.00017023277100312048, 5.072298516023148e-5, 4.8410170586051544e-8], [0.009085520703749992, 0.009085520654284349, 3.1570393442004007e-7, 3.157039163678158e-7], [1.5298540246266746e-6]], Any[Any[TensorMap((Rep[SU₂](0=>2, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>2, 1/2=>2, 1=>1)') ← ProductSpace(Rep[SU₂](0=>1))):
* Data for fusiontree FusionTree{Irrep[SU₂]}((1/2, 1/2), 0, (false, true), ()) ← FusionTree{Irrep[SU₂]}((0,), 0, (false,),

In [9]:
check_truncated_decomp_error=false;

euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined=combine_singlespin_sector(euR_set,evL_set,evR_set,SPIN_eig_set,true);
euR_set_grouped,evL_set_grouped,evR_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig=group_singlespin_sector(group_size,euR_set_combined,evL_set_combined,evR_set_combined,SPIN_eig_set_combined,true,U_fuse_chichi)
display("group information:")
display(DTrun_FLR_eig)

eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined=combine_singlespin_sector(eur_set,evl_set,evr_set,spin_eig_set,true)
eur_set_grouped,evl_set_grouped,evr_set_grouped,spin_eig_set_grouped,Dtrun_GLR_eig=group_singlespin_sector(group_size,eur_set_combined,evl_set_combined,evr_set_combined,spin_eig_set_combined,false,[])
display("group information:")
display(Dtrun_GLR_eig)


S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined=combine_singlespin_sector(S_set,Vh_set,U_set,SPIN_svd_set,false)
S_set_grouped,Vh_set_grouped,U_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd=group_singlespin_sector(group_size,S_set_combined,Vh_set_combined,U_set_combined,SPIN_svd_set_combined,true,U_fuse_chichi)
display("group information:")
display(DTrun_FLR_svd)

s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined=combine_singlespin_sector(s_set,vh_set,u_set,spin_svd_set,false)
s_set_grouped,vh_set_grouped,u_set_grouped,spin_svd_set_grouped,Dtrun_GLR_svd=group_singlespin_sector(group_size,s_set_combined,vh_set_combined,u_set_combined,spin_svd_set_combined,false,[])
display("group information:")
display(Dtrun_GLR_svd)



if check_truncated_decomp_error

    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_eig")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end

    M_temp=M*0;
    for cc=1:length(euR_set)
        euR=euR_set_combined[cc];
        evR=evR_set_combined[cc];
        evL=evL_set_combined[cc];
        spin=SPIN_eig_set_combined[cc]
        if euR==[]
        else
            euR0=euR
            for ccc=1:pow-1
                euR=euR*euR0;
            end
            M_temp=M_temp+evR*euR*evL*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(euR_set)
        euRs=euR_set_grouped[cc];
        evRs=evR_set_grouped[cc];
        evLs=evL_set_grouped[cc];
        spin=SPIN_eig_set_grouped[cc]
        if euRs==[]
        else
            for cs=1:length(euRs)
                euR0=euRs[cs];
                euR=euR0;
                for ccc=1:pow-1
                    euR=euR*euR0;
                end
                @tensor evR[:]:=U_fuse_chichi[-2,1,2]*evRs[cs][-1,1,2,-3,-4];
                evR=permute(evR,(1,2,3,),(4,));
                @tensor evL[:]:=evLs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                evL=permute(evL,(1,),(2,3,4,));
                M_temp1=M_temp1+evR*euR*evL*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_eig")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end

    M_temp=M*0;
    for cc=1:length(eur_set)
        eur=eur_set_combined[cc];
        evl=evl_set_combined[cc];
        evr=evr_set_combined[cc];
        spin=spin_eig_set_combined[cc]
        if eur==[]
        else
            eur0=eur;
            for ccc=1:pow-1
                eur=eur*eur0;
            end
            M_temp=M_temp+evr*eur*evl*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(eur_set)
        eurs=eur_set_grouped[cc];
        evrs=evr_set_grouped[cc];
        evls=evl_set_grouped[cc];
        spin=spin_eig_set_grouped[cc]
        if eurs==[]
        else
            for cs=1:length(eurs)
                eur0=eurs[cs];
                eur=eur0;
                for ccc=1:pow-1
                    eur=eur*eur0;
                end

                M_temp1=M_temp1+evrs[cs]*eur*evls[cs]*(spin*2+1);
            end
        end
    end
    display(norm(M-M_temp1)/norm(M))




    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check FLR_svd")
    @tensor M[:]:=Ag'[-1,-4,1]*OO[-2,2,-5,1]*Ag[-3,-6,2];
    M=permute(M,(1,2,3,),(4,5,6,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set[cc];
        U=U_set[cc];
        Vh=Vh_set[cc];
        spin=SPIN_svd_set[cc]
        for cs=1:length(S)
            M_temp=M_temp+S[cs]*U[cs]*Vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp=M*0;
    for cc=1:length(S_set)
        S=S_set_combined[cc];
        U=U_set_combined[cc];
        Vh=Vh_set_combined[cc];
        spin=SPIN_svd_set_combined[cc]
        if S==[]
        else
            M_temp=M_temp+U*S*Vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(S_set)
        Ss=S_set_grouped[cc];
        Us=U_set_grouped[cc];
        Vhs=Vh_set_grouped[cc];
        spin=SPIN_svd_set_grouped[cc]
        if Ss==[]
        else
            for cs=1:length(Ss)
                S=Ss[cs];
                @tensor U[:]:=U_fuse_chichi[-2,1,2]*Us[cs][-1,1,2,-3,-4];
                U=permute(U,(1,2,3,),(4,));
                @tensor Vh[:]:=Vhs[cs][-1,-2,1,2,-4]*U_fuse_chichi'[1,2,-3];
                Vh=permute(Vh,(1,),(2,3,4,));
                M_temp1=M_temp1+U*S*Vh*(spin*2+1);
            end
        end
    end
    display(norm(M-M_temp1)/norm(M))



    display("%%%%%%%%%%%%%%%%%%%%%%")
    display("check GLR_svd")
    @tensor M[:]:=Ag'[-1,-3,1]*Ag[-2,-4,1];
    M=permute(M,(1,2,),(3,4,));
    M0=M;
    for cc=1:pow-1
        M=M*M0;
    end
    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set[cc];
        u=u_set[cc];
        vh=vh_set[cc];
        spin=spin_svd_set[cc]
        for cs=1:length(s)
            M_temp=M_temp+s[cs]*u[cs]*vh[cs]*(spin[cs]*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp=M*0;
    for cc=1:length(s_set)
        s=s_set_combined[cc];
        vh=vh_set_combined[cc];
        u=u_set_combined[cc];
        spin=spin_svd_set_combined[cc]
        if s==[]
        else
            M_temp=M_temp+u*s*vh*(spin*2+1);
        end
    end
    display(norm(M-M_temp)/norm(M))

    M_temp1=M_temp*0;
    for cc=1:length(s_set)
        ss=s_set_grouped[cc];
        us=u_set_grouped[cc];
        vhs=vh_set_grouped[cc];
        spin=spin_svd_set_grouped[cc]
        if ss==[]
        else
            for cs=1:length(ss)
                s=ss[cs];
                M_temp1=M_temp1+us[cs]*s*vhs[cs]*(spin*2+1);
            end
        end
    end

    display(norm(M-M_temp1)/norm(M))
end

"group information:"

7-element Vector{Any}:
 UnitRange{Int64}[1:123]
 UnitRange{Int64}[1:185]
 UnitRange{Int64}[1:170]
 UnitRange{Int64}[1:110]
 UnitRange{Int64}[1:51]
 UnitRange{Int64}[1:12]
 ComplexF64[]

"group information:"

5-element Vector{Any}:
 UnitRange{Int64}[1:9]
 UnitRange{Int64}[1:12]
 UnitRange{Int64}[1:9]
 UnitRange{Int64}[1:4]
 UnitRange{Int64}[1:1]

"group information:"

7-element Vector{Any}:
 UnitRange{Int64}[1:113]
 UnitRange{Int64}[1:168]
 UnitRange{Int64}[1:150]
 UnitRange{Int64}[1:94]
 UnitRange{Int64}[1:40]
 UnitRange{Int64}[1:10]
 Float64[]

"group information:"

5-element Vector{Any}:
 UnitRange{Int64}[1:9]
 UnitRange{Int64}[1:12]
 UnitRange{Int64}[1:9]
 UnitRange{Int64}[1:4]
 UnitRange{Int64}[1:1]

In [10]:
        
function excitation_TrunTransOp_iterative_H_eff(x,input_transform,output_transform,A1,A2,Ag,pow,VR,EU,VL,SPIN,N,k,DTrun_list)
    @assert pow==round((N-2)/2)
    x=input_transform*x;# do this calculation on cpu because the matrix 'input_transform' maybe large
    x=permute(x,(1,2,3,4,),());
    H_eff_x=x*0;
    cm=1;

    for cn=1:N
            coe=exp(-im*k*(cm-cn))
            H_eff_x0=H_eff_x*0;
            for c_sector=1:length(DTrun_list)
                for  c_comp=1:length(DTrun_list[c_sector])
                    #display(DTrun_list[c_sector][c_comp])
                    
                    H_eff_x0=H_eff_x0+coe*contract_H_eff_trun(x,A1,A2,Ag,VR[c_sector][c_comp],EU[c_sector][c_comp],VL[c_sector][c_comp],SPIN[c_sector],cm,cn,pow,N,DTrun_list[c_sector][c_comp]);
                end
            end
            H_eff_x=H_eff_x+H_eff_x0
    end
    x=permute(H_eff_x,(1,2,3,),(4,));
    x_output=output_transform*x;
    return x_output
end

function excitation_TrunTransOp_iterative_norm_eff(Ag,pow,N,k)
    @assert pow==round((N-2)/2)
    norm_eff=permute(TensorMap(randn,codomain(Ag)←codomain(Ag))*0,(1,2,3,4,5,6,),());
    cm=1;
    for cn=1:N
            coe=exp(-im*k*(cm-cn))
            norm_eff=norm_eff+coe*contract_norm_eff(Ag,cm,cn,N)
    end
    return norm_eff
end


excitation_TrunTransOp_iterative_norm_eff (generic function with 1 method)

In [11]:
function v_Transop(VL_comp,Ag,A1,A2,L)
    VL_comp=copy(VL_comp)
    for cl=1:L
        @tensor VL_comp[:]:=VL_comp[-1,1,3,5,7]*Ag'[1,-2,2]*A2[3,4,-3,2]*A1[5,6,-4,4]*Ag[7,-5,6];
    end
    return VL_comp
end



v_Transop (generic function with 1 method)

In [12]:
function contract_H_eff_trun(x,A1,A2,Ag,VR_comp,EU_comp,VL_comp,spin_comp,cm,cn,pow,N,D_set)
    H_eff_x=x*0;
    L_Ea=cn-cm-1
    L_Eb=N-cn
    if L_Ea>=L_Eb
        L=L_Ea
        decomp_posit="Ea"
    elseif L_Ea<L_Eb
        L=L_Eb
        decomp_posit="Eb"
    end
    
    VL_comp=v_Transop(VL_comp,Ag,A1,A2,L-pow);
    VR_comp=VR_comp*EU_comp;
    

    
    if cm==cn # cm=cn=0
        @tensor ttt[:]:=VL_comp[9,-1,5,3,1]*A2[5,4,6,-3]*A1[3,2,7,4]*x[1,8,2,-4]*VR_comp[-2,6,7,8,9];
        H_eff_x=H_eff_x+ttt;
    elseif cn==2
        @tensor T1[:]:=VL_comp[-1,-2,5,3,1]*A2[5,4,-3,-6]*A1[3,2,-4,4]*Ag[1,-5,2];
        @tensor T2[:]:=Ag'[-1,1,2]*A2[-2,4,3,2]*A1[-3,-5,5,4]*VR_comp[1,3,5,-4,-6];
        @tensor T1T2x[:]:=T1[6,-1,3,4,5,-3]*T2[-2,3,4,2,1,6]*x[5,2,1,-4];
        H_eff_x=H_eff_x+T1T2x;
    elseif cn==N
        @tensor T1[:]:=VL_comp[-1,1,3,5,-2]*Ag'[1,-3,2]*A2[3,4,-4,2]*A1[5,-6,-5,4];
        @tensor T2[:]:=A2[-1,4,5,-5]*A1[-2,2,3,4]*Ag[-3,1,2]*VR_comp[-4,5,3,1,-6];
        @tensor T1xT2[:]:=T1[6,1,-1,3,4,2]*x[1,5,2,-4]*T2[3,4,5,-2,-3,6];
        H_eff_x=H_eff_x+T1xT2;
    else
        if  decomp_posit=="Ea"
            @tensor T1[:]:=VL_comp[-1,1,3,5,7]*Ag'[1,-2,2]*A2[3,4,-3,2]*A1[5,6,-4,4]*x[7,-5,6,-6];
            @tensor T2[:]:=A2[-1,4,5,-5]*A1[-2,2,3,4]*Ag[-3,1,2]*VR_comp[-4,5,3,1,-6];
            @assert L_Eb>=1
            for cr=1:L_Eb
                @tensor T1[:]:=T1[-1,1,3,5,7,-6]*Ag'[1,-2,2]*A2[3,4,-3,2]*A1[5,6,-4,4]*Ag[7,-5,6];
            end
            @tensor T1T2[:]:=T1[4,-1,1,2,3,-4]*T2[1,2,3,-2,-3,4];
            H_eff_x=H_eff_x+T1T2;
        elseif decomp_posit=="Eb"
            @tensor T1[:]:=VL_comp[-1,-2,5,3,1]*A2[5,4,-3,-6]*A1[3,2,-4,4]*Ag[1,-5,2];
            @tensor T2[:]:=Ag'[-1,1,2]*A2[-2,4,3,2]*A1[-3,6,5,4]*x[-4,7,6,-6]*VR_comp[1,3,5,7,-5];
            @assert L_Ea>=1
            for cr=1:L_Ea
                @tensor T2[:]:=Ag'[-1,1,2]*A2[-2,4,3,2]*A1[-3,6,5,4]*Ag[-4,7,6]*T2[1,3,5,7,-5,-6];
            end
            @tensor T1T2[:]:=T1[4,-1,1,2,3,-3]*T2[-2,1,2,3,4,-4];
            H_eff_x=H_eff_x+T1T2;
        end
    end
    H_eff_x=H_eff_x*(2*spin_comp+1);
    return H_eff_x
end


    

contract_H_eff_trun (generic function with 1 method)

In [13]:
function contract_norm_eff(AL,cm,cn,N)

    mps=create_puMPS(AL,N);
    mps1=mps;
    mps2=mps;
    if cm==cn # cm=cn=1
        E=contiguous_TM(mps1,mps2,2:N)
        Id=id(space(Ag,3));
        @tensor norm_eff[:]:=E[-2,-5,-1,-4]*Id[-3,-6];
    elseif cn==2
        E=contiguous_TM(mps1,mps2,3:N);
        @tensor norm_eff[:]:=mps2[1][1,-4,-3]*mps1[2]'[-2,2,-6]*E[2,-5,-1,1];
    elseif cn==N
        E=contiguous_TM(mps1,mps2,2:N-1);
        @tensor norm_eff[:]:=mps2[1][-5,1,-3]*E[-2,1,2,-4]*mps1[N]'[2,-1,-6]
    else
        T1=mps2[1];
        T2=contiguous_TM(mps1,mps2,2:(cn-1))
        T3=mps1[cn]';
        T4=contiguous_TM(mps1,mps2,(cn+1):N)
        @tensor norm_eff[:]:=T1[6,1,-3]*T2[-2,1,3,-4]*T3[3,4,-6]*T4[4,-5,-1,6];
    end
    return norm_eff
end



contract_norm_eff (generic function with 1 method)

In [14]:
function contiguous_TM(mps1,mps2,positions)
    if length(positions)==1
        @tensor E[:]:=mps1[positions[1]]'[-1,-3,3]*mps2[positions[1]][-2,-4,3];
    elseif length(positions)>1
        @tensor E[:]:=mps1[positions[1]]'[-1,-3,3]*mps2[positions[1]][-2,-4,3];
        for ci=positions[2]:positions[end]
            @tensor E[:]:=E[-1,-2,1,2]*mps1[ci]'[1,-3,3]*mps2[ci][2,-4,3];
        end
    end
    return E
end

contiguous_TM (generic function with 1 method)

In [15]:

function create_puMPS(A,N)
    mps=Vector{Any}(undef, N);
    for c1=1:N
        mps[c1]=A
    end
    return mps
end

create_puMPS (generic function with 1 method)

In [16]:
function solve_ITEBD_excitation_TrunTransOp_iterative(Ag,A1,A2,n_E,N,kset,ES_sectors,pow,U,S,Vt,SPIN_group,DTrun_group)
    
    Eset=Matrix{Any}(undef, length(kset),length(ES_sectors)); 

    for kk=1:length(kset)

        ck=kset[kk]
        k=2*pi*ck/N
        

        norm_eff=excitation_TrunTransOp_iterative_norm_eff(Ag,pow,N,k) # put it on cpu because this matrix maybe large
        norm_eff=permute(norm_eff,(1,2,3,),(4,5,6,))
        uu,ss,vvt=tsvd(norm_eff, trunc=truncerr(0.0000001));
        norm_eff=[]#clear this big matrix
        input_transform=vvt';
        output_transform=uu';
        output_transform=output_transform;
        output_transform=pinv(ss)*output_transform;

        for sector_ind=1:length(ES_sectors)
            SPIN=ES_sectors[sector_ind];
            sectr=Irrep[SU₂](SPIN);
            display("sector "*"k="*string(ck)*", spin="*string(SPIN)*":");

            v_init=TensorMap(randn,domain(input_transform), SU₂Space(SPIN=>1));
            
            excitation_iterative(x)=excitation_TrunTransOp_iterative_H_eff(x,input_transform,output_transform,A1,A2,Ag,pow,U,S,Vt,SPIN_group,N,k,DTrun_group)
            
            @time Es,_=eigsolve(excitation_iterative, v_init, n_E,:LM,Arnoldi());
            Eset[kk,sector_ind]=Es
            display(Es)
        end

    end


    return kset, Eset
end


solve_ITEBD_excitation_TrunTransOp_iterative (generic function with 1 method)

In [17]:
for cc=1:length(S_set_grouped)
    for cc2=1:length(S_set_grouped[cc])
        display(diag(convert(Array,S_set_grouped[cc][cc2])))
    end
end

113-element Vector{ComplexF64}:
 0.00011301829795290723 + 0.0im
    4.99324262734249e-5 + 0.0im
   3.739665388913006e-5 + 0.0im
   2.829793213824554e-5 + 0.0im
  1.6814167642705153e-5 + 0.0im
  1.2728092995973736e-5 + 0.0im
  1.0211751279340665e-5 + 0.0im
   6.069509196382349e-6 + 0.0im
   5.862917250856472e-6 + 0.0im
   4.082454915480337e-6 + 0.0im
                        ⋮
 1.6974738062064808e-11 + 0.0im
 1.3015697099386775e-11 + 0.0im
 1.2153015608207201e-11 + 0.0im
  8.923156135858704e-12 + 0.0im
  5.172911765089699e-12 + 0.0im
  3.952371582007857e-12 + 0.0im
 2.2595838025825315e-12 + 0.0im
 2.0641207021487445e-12 + 0.0im
   1.31383311107688e-12 + 0.0im

336-element Vector{ComplexF64}:
   7.218536084867458e-5 + 0.0im
   7.218536084867458e-5 + 0.0im
   6.206621833002712e-5 + 0.0im
   6.206621833002712e-5 + 0.0im
  3.5439588180019234e-5 + 0.0im
  3.5439588180019234e-5 + 0.0im
   2.582785550553047e-5 + 0.0im
   2.582785550553047e-5 + 0.0im
   2.014792298765307e-5 + 0.0im
   2.014792298765307e-5 + 0.0im
                        ⋮
  3.426168261612876e-12 + 0.0im
  2.982903002621745e-12 + 0.0im
  2.982903002621745e-12 + 0.0im
 2.3059957401297655e-12 + 0.0im
 2.3059957401297655e-12 + 0.0im
 2.0614081542132084e-12 + 0.0im
 2.0614081542132084e-12 + 0.0im
 1.3533931586647562e-12 + 0.0im
 1.3533931586647562e-12 + 0.0im

450-element Vector{ComplexF64}:
   3.862857348165784e-5 + 0.0im
   3.862857348165784e-5 + 0.0im
   3.862857348165784e-5 + 0.0im
  1.9555420572725516e-5 + 0.0im
  1.9555420572725516e-5 + 0.0im
  1.9555420572725516e-5 + 0.0im
  1.8836452501910457e-5 + 0.0im
  1.8836452501910457e-5 + 0.0im
  1.8836452501910457e-5 + 0.0im
  1.2172127864861267e-5 + 0.0im
                        ⋮
 3.6663445749087194e-12 + 0.0im
 3.6663445749087194e-12 + 0.0im
 3.6663445749087194e-12 + 0.0im
 2.6413993705092935e-12 + 0.0im
 2.6413993705092935e-12 + 0.0im
 2.6413993705092935e-12 + 0.0im
  2.087222130314583e-12 + 0.0im
  2.087222130314583e-12 + 0.0im
  2.087222130314583e-12 + 0.0im

376-element Vector{ComplexF64}:
   9.459008905312802e-6 + 0.0im
   9.459008905312802e-6 + 0.0im
   9.459008905312802e-6 + 0.0im
   9.459008905312802e-6 + 0.0im
   5.588496099109405e-6 + 0.0im
   5.588496099109405e-6 + 0.0im
   5.588496099109405e-6 + 0.0im
   5.588496099109405e-6 + 0.0im
  1.7760148840329521e-6 + 0.0im
  1.7760148840329521e-6 + 0.0im
                        ⋮
 2.7089848304215934e-12 + 0.0im
  1.836788275635688e-12 + 0.0im
  1.836788275635688e-12 + 0.0im
  1.836788275635688e-12 + 0.0im
  1.836788275635688e-12 + 0.0im
 1.4363467784390902e-12 + 0.0im
 1.4363467784390902e-12 + 0.0im
 1.4363467784390902e-12 + 0.0im
 1.4363467784390902e-12 + 0.0im

200-element Vector{ComplexF64}:
  1.0604409686539363e-7 + 0.0im
  1.0604409686539363e-7 + 0.0im
  1.0604409686539363e-7 + 0.0im
  1.0604409686539363e-7 + 0.0im
  1.0604409686539363e-7 + 0.0im
   8.191026888469778e-8 + 0.0im
   8.191026888469778e-8 + 0.0im
   8.191026888469778e-8 + 0.0im
   8.191026888469778e-8 + 0.0im
   8.191026888469778e-8 + 0.0im
                        ⋮
 1.4462138086182732e-12 + 0.0im
 1.4462138086182732e-12 + 0.0im
 1.4462138086182732e-12 + 0.0im
 1.4462138086182732e-12 + 0.0im
 1.1585540107354422e-12 + 0.0im
 1.1585540107354422e-12 + 0.0im
 1.1585540107354422e-12 + 0.0im
 1.1585540107354422e-12 + 0.0im
 1.1585540107354422e-12 + 0.0im

60-element Vector{ComplexF64}:
   8.40230665584983e-10 + 0.0im
   8.40230665584983e-10 + 0.0im
   8.40230665584983e-10 + 0.0im
   8.40230665584983e-10 + 0.0im
   8.40230665584983e-10 + 0.0im
   8.40230665584983e-10 + 0.0im
  5.835104610050267e-10 + 0.0im
  5.835104610050267e-10 + 0.0im
  5.835104610050267e-10 + 0.0im
  5.835104610050267e-10 + 0.0im
                        ⋮
 5.6459897759301425e-12 + 0.0im
 5.6459897759301425e-12 + 0.0im
 5.6459897759301425e-12 + 0.0im
 4.6909670570871725e-12 + 0.0im
 4.6909670570871725e-12 + 0.0im
 4.6909670570871725e-12 + 0.0im
 4.6909670570871725e-12 + 0.0im
 4.6909670570871725e-12 + 0.0im
 4.6909670570871725e-12 + 0.0im

In [24]:
Dtrun_method="svds";
ES_sectors=[0,1/2,1,3/2];
kset=0:N-1;
#kset=0:0
Eset=[];
Trun_err=0;
DTrun=0;
print("calculate ES for N="*string(N));
display("kset="*string(kset));
pow=round((N-2)/2);



if Dtrun_method=="eigs"
    DTrun=length(group_numbers(SPIN_eig_set));
    display("DTrun="*string(DTrun));

    euRs=abs.(group_numbers(euR_set));
    Trun_err=(minimum(euRs)/maximum(euRs))^pow;


    euR_pow=deepcopy(euR_set_grouped);
    for ca=1:length(euR_pow)
        for cb=1:length(euR_pow[ca])
            euR_pow[ca][cb]=euR_pow[ca][cb]^Int(pow);
        end
    end

    H_renorm=maximum(euRs);

    kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1/H_renorm,O2,EH_n,N,kset,ES_sectors,pow,evR_set_grouped,euR_pow,evL_set_grouped,SPIN_eig_set_grouped,DTrun_FLR_eig)

elseif Dtrun_method=="svds"
    DTrun=length(group_numbers(SPIN_svd_set));
    display("DTrun="*string(DTrun));

    Ss=abs.(group_numbers(S_set));
    Trun_err=(minimum(Ss)/maximum(Ss));

    H_renorm=maximum(Ss)^(1/pow);

    kset,Eset=solve_ITEBD_excitation_TrunTransOp_iterative(Ag,O1/H_renorm,O2,EH_n,N,kset,ES_sectors,pow,U_set_grouped,S_set_grouped,Vh_set_grouped,SPIN_svd_set_grouped,DTrun_FLR_svd)
end

ES_filenm="ES_"*Dtrun_method*"_D"*string(D)*"_chi"*string(chi)*"_W"*string(W)*"_N"*string(N)*".mat";
matwrite(ES_filenm, Dict(
    "kset" => convert(Vector,kset),
    "ES_sectors" => ES_sectors,
    "Eset" => Eset,
    "Trun_err"=>Trun_err,
    "DTrun"=>DTrun
); compress = false)


"kset=0:7"

"DTrun=575"

"sector k=0, spin=0.0:"

13-element Vector{ComplexF64}:
     6.16067952730054e-8 + 2.7319102775571644e-8im
    6.154389901857716e-8 - 2.7112595880009944e-8im
   1.7767981930448177e-8 - 4.508673789237453e-9im
   1.7571230549741326e-8 + 4.357283711331697e-9im
    8.103256870817949e-9 + 1.2250578212880684e-11im
   7.2171717771447635e-9 - 5.460071015922155e-12im
    6.721551859353368e-9 + 2.1653909302454024e-11im
    4.614322284261491e-9 - 7.496580093262892e-10im
    4.575836227010203e-9 + 7.016795191195576e-10im
    3.712939014179335e-9 + 9.861534093843536e-12im
   1.0923500278541706e-9 + 7.553410603168624e-12im
   6.301027333744303e-10 + 4.647888374985662e-12im
 -2.2538083243157266e-10 - 3.7877408363559146e-11im

"sector k=0, spin=0.5:"

...

10-element Vector{ComplexF64}:
  2.4071146602023272e-8 + 5.663343361690046e-9im
   2.398055735131174e-8 - 5.712930730589758e-9im
  5.3704459038163294e-9 + 6.820637041209821e-10im
   5.369813617914417e-9 - 6.751768492728152e-10im
  3.1892720779492723e-9 - 3.1998116172301563e-10im
   3.189715423015714e-9 + 2.980025634786285e-10im
  1.9601586044604606e-9 - 3.563308535023325e-10im
  1.9484167024226416e-9 + 3.5489772998109303e-10im
  1.3172567045273198e-9 + 3.710611860378393e-12im
 3.6982321685233897e-10 - 1.2594998532174725e-11im

"sector k=2, spin=0.0:"

12-element Vector{ComplexF64}:
 3.7929608023151406e-8 - 8.639530800437376e-11im
 2.7569674987899228e-8 + 1.2440367886584145e-10im
 1.2301304269421864e-8 - 3.054709593227467e-11im
  9.215061622078263e-9 - 6.98294064351257e-12im
  4.889995099221981e-9 + 8.618173336699389e-12im
  4.250703857887253e-9 - 3.292286056972736e-11im
 3.1568295807713814e-9 - 3.4703355332113904e-10im
 3.1360255550816147e-9 + 3.633064977793279e-10im
  2.305959334465258e-9 - 6.463098341204082e-12im
 2.1214603286719634e-9 - 7.733302185482768e-10im
  2.070791951249029e-9 + 7.499884565119048e-10im
 -6.82564546415699e-10 - 3.863987737324195e-11im

"sector k=2, spin=0.5:"

18-element Vector{ComplexF64}:
   4.64473050348026e-8 - 5.3331879438055146e-11im
  3.323660400469344e-8 + 1.5972667075286246e-11im
 1.7639164248285328e-8 + 8.434704882769119e-12im
 1.1545679919761759e-8 - 3.3985803348217554e-9im
  1.148654842185733e-8 + 3.41018420119485e-9im
  8.927108443525264e-9 - 2.431000507945682e-11im
  6.688980240216049e-9 + 5.795125581390506e-12im
  4.952606861454793e-9 + 2.912428531774868e-12im
  4.048728785720595e-9 - 1.3126476927171353e-9im
  4.040329272673952e-9 + 1.312754539396861e-9im
 3.0225621119816196e-9 - 6.390691325222207e-11im
   2.86169968166638e-9 - 4.2293864597070855e-11im
   2.60743492620942e-9 + 5.304650024844781e-10im
 2.5686294217154997e-9 - 4.627995994215127e-10im
 1.9268749315803528e-9 - 6.012857048767579e-10im
  1.902824695387694e-9 + 5.840714340689853e-10im
 1.3008358708014516e-9 + 5.60496040714044e-12im
 3.240845086300749e-10 - 1.362790753412335e-12im